In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from matplotlib import pyplot as plt
%matplotlib inline

2024-07-14 20:04:28.562239: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-14 20:04:28.792172: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-14 20:04:29.050242: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-14 20:04:29.300022: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-14 20:04:29.359055: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-14 20:04:29.731842: I tensorflow/core/platform/cpu_feature_gu

In [2]:
df = pd.read_csv("insurance_data.csv")
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


**Train test split**

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age','affordibility']],df.bought_insurance,test_size=0.2, random_state=25)

Preprcessing the data so that age and affordibility are in same scaling range.

In [4]:
X_train_scaled=X_train.copy()
X_train_scaled['age']=X_train_scaled['age']/100

X_test_scaled=X_test.copy()
X_test_scaled['age']=X_test_scaled['age']/100

In [5]:
X_train_scaled.head()

,age,affordibility
0,0.22,1
13,0.29,0
6,0.55,0
17,0.58,1
24,0.50,1


### Usuing tensorflow

In [ ]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5000)

In [12]:
model.evaluate(X_test_scaled,y_test)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 203ms/step - accuracy: 1.0000 - loss: 0.3550


[0.3549776077270508, 1.0]

In [17]:
y_test

2     1
10    0
21    0
11    0
14    1
9     1
Name: bought_insurance, dtype: int64

In [13]:
model.predict(X_test_scaled)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step


array([[0.7054848 ],
       [0.3556957 ],
       [0.16827832],
       [0.47801185],
       [0.7260696 ],
       [0.8294983 ]], dtype=float32)

get the value of weights and bias

In [19]:
coef, intercept = model.get_weights()
coef, intercept

(array([[5.060863 ],
        [1.4086521]], dtype=float32),
 array([-2.913703], dtype=float32))

$y=w1*X1+w2*X2+bias$<br>
$y=predicted value$<br>
$X1=Age$<br>
$X2=Affordibility$

from the model weights and bias are calculated<br>
$w1=5.060863$<br>
$w2=1.4086521$<br>
$bias=-2.913703$

sigmoid<br>
$\sigma(x) = \frac{1}{1 + e^{-x}}$

In [20]:
def sigmoid(x):
    import math
    return 1 / (1 + math.exp(-x))
sigmoid(18)

0.9999999847700205

instead of using model.predict(X_test_scaled) we create our own pred_func with prev calculated weights and bias<br>
note: in X_test_scaled we have 2 columns age and affordibility<br>

$y=5.060863*X1+1.4086521*X2+(-2.913703)$

In [21]:
def prediction_function(age, affordibility):
    weighted_sum = coef[0]*age + coef[1]*affordibility + intercept
    return sigmoid(weighted_sum)

In [22]:
prediction_function(.47, 1)

/tmp/ipykernel_23832/1668309685.py:3: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  return 1 / (1 + math.exp(-x))


0.705484819775958

### Lets create gradient descent function from scratch

first create helper functions like loss function, activation function

### log_loss function

$LogLoss = -\frac{1}{n} \sum_{i=1}^{n} y_i \log(\hat{y}_i) + (1-y_i) \log(1-\hat{y}_i)$

In [23]:
def log_loss(y_true,y_predicted):
    epsilon=1e-15
    y_predicted_new=[max(i,epsilon) for i in y_predicted]
    y_predicted_new=[min(i,1-epsilon) for i in y_predicted_new]
    y_predicted_new=np.array(y_predicted_new)
    return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

### np_sigmoid function

In [24]:
def np_sigmoid(x):
    # use np instead of math because it can handle arrays
    return 1 / (1 + np.exp(-x))
np_sigmoid(np.array([12,0,1]))

array([0.99999386, 0.5       , 0.73105858])

### [gradient_descent function](https://towardsdatascience.com/gradient-descent-explained-9b953fc0d2c)

$w_{1} = w_{1} - lr * \frac{\partial L}{\partial w}$<br>

$w_{2} = w_{2} - lr * \frac{\partial L}{\partial w}$<br>

$\frac{\partial L}{\partial w} = \frac{1}{n} \sum_{i=1}^{n} x_i(\hat{y} - y_i)$<br>

$b = b - lr * \frac{\partial L}{\partial b}$<br>

$\frac{\partial L}{\partial b} = \frac{1}{n} \sum_{i=1}^{n} (\hat{y} - y_i)$<br>

$\hat{y} = predicted value$<br>
$\hat{y} = \sigma(w_{i} * x_{i} + b)$<br>
$y = actual value$

In [25]:
def gradient_descent(age, affordability, y_true, epochs, loss_thresold):
    w1 = w2 = 1
    bias = 0
    rate = 0.5
    n = len(age)
    for i in range(epochs):
        weighted_sum = w1 * age + w2 * affordability + bias
        y_predicted = np_sigmoid(weighted_sum)
        loss = log_loss(y_true, y_predicted)

        w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true)) 
        w2d = (1/n)*np.dot(np.transpose(affordability),(y_predicted-y_true)) 

        bias_d = np.mean(y_predicted-y_true)
        w1 = w1 - rate * w1d
        w2 = w2 - rate * w2d
        bias = bias - rate * bias_d

        print (f'Epoch:{i}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}')

        if loss<=loss_thresold:
            break

    return w1, w2, bias

In [ ]:
gradient_descent(X_train_scaled['age'],X_train_scaled['affordibility'],y_train,1000, 0.4631)

In [27]:
w1=5.051047623653049
w2=1.4569794548473887
b= -2.9596534546250037

In [35]:
def prediction_function(age, affordibility):
    weighted_sum = w1*age + w2*affordibility + b
    return np_sigmoid(weighted_sum)

In [43]:
age = np.array(X_test_scaled['age'])
affordibility = np.array(X_test_scaled['affordibility'])
(age,affordibility)

(array([0.47, 0.18, 0.26, 0.28, 0.49, 0.61]), array([1, 1, 0, 1, 1, 1]))

In [48]:
test_result=prediction_function(age, affordibility)

In [53]:
new_test_result=test_result.copy()
new_test_result[new_test_result>=0.5]=1
new_test_result[new_test_result<0.5]=0
new_test_result.astype(int)

array([1, 0, 0, 0, 1, 1])

In [54]:
y_test.shape, new_test_result.shape

((6,), (6,))

In [59]:
def accuracy(y_true,y_predicted):
    return np.sum(y_true==y_predicted)/len(y_true)*100

In [60]:
accuracy(np.array(y_test),new_test_result)

100.0

**Note: I deleted some of cell outputs to make it more readable**